In [1]:
#instaaling the fastText
#!pip install fastText

     |████████████████████████████████| 68 kB 1.1 MB/s 
  Using cached pybind11-2.8.1-py2.py3-none-any.whl (208 kB)
  Created wheel for fastText: filename=fasttext-0.9.2-cp37-cp37m-linux_x86_64.whl size=3123584 sha256=2a2377a13c55615933cea18067f1c143cbf87ef0534c5f9ba92d52b61b394610
  Stored in directory: /root/.cache/pip/wheels/4e/ca/bf/b020d2be95f7641801a6597a29c8f4f19e38f9c02a345bab9b
Successfully built fastText


### importing the libraries

In [3]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer, TfidfTransformer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.pipeline import Pipeline
from sklearn.decomposition import PCA
from sklearn.preprocessing import normalize
from sklearn.metrics import pairwise_distances
from gensim.utils import simple_preprocess
import csv
import fasttext
from fasttext import train_supervised 

In [6]:
%%time

'''
importing the data of genre and plot and most frequent words in the plot column
making a dict for each genre of most common words
then splitting the data into Train and test 
and making a function that trims the plot column using the most common word on the training data
'''
df=pd.read_csv('wiki_movie_plots_deduped_cleaned_genre_and_plot.csv')
data=pd.read_csv("most_frequent_words_30.csv")
dict_words={i:(data[data['Genre']==i]['Words']).to_list() for i in data['Genre'].unique()}
def reduce_words(x):
    return ' '.join([i for i in x['Plot_cleanned'].split() if i in  dict_words[x['Genre_updated']]])
    
Train=df[~pd.isnull(df['Genre_updated'])]
Test=df[pd.isnull(df['Genre_updated'])]
Train['Plot_trimmed']=Train[['Genre_updated','Plot_cleanned']].apply(lambda x: reduce_words(x),axis=1)
Train.drop_duplicates(subset='Plot_trimmed',inplace=True) #removing the duplicate columns


CPU times: user 36.8 s, sys: 290 ms, total: 37.1 s
Wall time: 37.2 s


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if __name__ == '__main__':
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # Remove the CWD from sys.path while we load stuff.


In [7]:
%%time
# splitting the data into train, val and test 

X_train=Train['Plot_trimmed']
y_train=Train['Genre_updated']


X_train, X_val, y_train, y_val = train_test_split(X_train,y_train,test_size=0.3,random_state=123,stratify=y_train)


X_test=Test['Plot_cleanned']
y_test=Test['Genre_updated']

CPU times: user 41.1 ms, sys: 1.98 ms, total: 43.1 ms
Wall time: 43.6 ms


In [32]:
# fastText is auto text classification model so for identifying the label we have to make all the classes/labels start with  '__label__'

Train['Genre_updated']=Train['Genre_updated'].apply(lambda x: '__label__' + x)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [49]:
#making the txt file for training and testing the fasttext model

train_file =  '/train.txt'
Train[:20000].to_csv(train_file, index = False, 
                                          sep = ' ',
                                          header = None, 
                                          quoting = csv.QUOTE_NONE, 
                                          quotechar = "", 
                                          escapechar = " ", columns=['Plot_trimmed','Genre_updated'] )

test_file = '/test.txt'
Train[20000:].to_csv(test_file,  index = False, 
                                          sep = ' ',
                                          header = None, 
                                          quoting = csv.QUOTE_NONE, 
                                          quotechar = "", 
                                          escapechar = " ", columns=['Plot_trimmed','Genre_updated'] )

In [52]:
# Evaluating performance on the entire test file 
model = train_supervised('/train.txt', wordNgrams = 2)
model.test('/test.txt')     

(12334, 0.7086914220852927, 0.7086914220852927)

In [55]:
# Evaluating performance on the entire test file with added parameters
model = train_supervised(input=train_file, label="__label__", lr=1.0, epoch=75, loss='ova', wordNgrams=2, dim=200, thread=2, verbose=100)
model.test(test_file)

(12334, 0.7101508026593157, 0.7101508026593157)

In [63]:
model.test_label(test_file)

{'__label__action': {'f1score': 1.7295873573309921,
  'precision': 0.8647936786654961,
  'recall': nan},
 '__label__adventure': {'f1score': 0.9897959183673469,
  'precision': 0.49489795918367346,
  'recall': nan},
 '__label__animated': {'f1score': 1.5666666666666667,
  'precision': 0.7833333333333333,
  'recall': nan},
 '__label__anime': {'f1score': 0.0, 'precision': 0.0, 'recall': nan},
 '__label__biography': {'f1score': 1.0666666666666667,
  'precision': 0.5333333333333333,
  'recall': nan},
 '__label__comedy': {'f1score': 1.2424489795918368,
  'precision': 0.6212244897959184,
  'recall': nan},
 '__label__crime': {'f1score': 1.3858267716535433,
  'precision': 0.6929133858267716,
  'recall': nan},
 '__label__documentary': {'f1score': 0.7547169811320755,
  'precision': 0.37735849056603776,
  'recall': nan},
 '__label__drama': {'f1score': 1.5018418815528478,
  'precision': 0.7509209407764239,
  'recall': nan},
 '__label__family': {'f1score': 1.2095238095238094,
  'precision': 0.60476190

In [70]:
#predicting on the single test data
model.predict(X_test[0])

(('__label__social',), array([1.00000034e-05]))